### Libraries & Configurations

In [13]:
import pandas as pd
pd.set_option('display.max_rows', None)   # Show all rows
pd.set_option('display.max_columns', None) # Show all columns
pd.set_option('display.width', None)      # Disable line wrapping
pd.set_option('display.max_colwidth', None) # Show long text fully

import re
import ast

import os

In [14]:
#-----------Data Paths--------------
FILE_RESERVATION = r"C:\Users\joannelee\Desktop\Personal\Xinyi_files\信義居家_一般訂單Reservation.csv"
FILE_UNORDERED = r"C:\Users\joannelee\Desktop\Personal\Xinyi_files\信義居家_工程訂單UnorderedReservation.csv"

In [15]:
df_reservation = pd.read_csv(FILE_RESERVATION)
df_unordered = pd.read_csv(FILE_UNORDERED)

c:\Users\joannelee\AppData\Local\anaconda3\envs\test_env\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (3,6,11,27,28,29,30,33,36,39,67) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


#### Select Dataset to clean and folder name

In [16]:
#-----選擇要處理的檔案-----
df_raw = df_unordered #df_reservation / df_unordered
folder_name = 'unordered' # reservation / unordered

### Functions

In [17]:
"""
不管是 dict 還是 list of dict 的欄位，在資料內都是以 str. 的型態儲存，因次需先轉換成對應的 python object
"""

def fix_missing_commas(raw_string):
    """
    注意到 list of dict 缺少 dict 之間的逗號，透過此function將逗號加在適當的位置
    Example: [{...}{...}] ➝ [{...}, {...}]
    """
    # Insert a comma between '}{' only if they are not already separated
    fixed_string = re.sub(r'\}[\s]*\{', '}, {', raw_string)
    return fixed_string


def parse_json_like(val):
    """
    將 string 讀成 Python 物件
    """
    try:
        if type(val) == type("hi"):
            val = fix_missing_commas(val)

            fixed = re.sub(r'\}[\s]*\{', '}, {', val)
            #處理時間資料
            fixed_time = re.sub(r'Timestamp\((.*?)\)', r'\1', fixed)
            #處理array型式
            cleaned = re.sub(r'array\((\[.*?\])\s*,\s*dtype=.*?\)', r'\1', fixed_time)

            return ast.literal_eval(cleaned)
        else:
            return val
    except Exception as e:
        #print(str_val)
        #print(f"Error parsing services : {e}")
        return val

In [18]:
#Dict 展開
def expand_dict_like(df, col_name, converted = False, rename = True):
    """
    讀入有 dict 型態的欄位，並回傳key-value pairs 提出後的dataframe 
    converted : 資料是否已轉換為 python objects
    rename : 是否修改欄位名稱為 [原始欄位名稱]_[key]
    """
    if not converted:
        df_parsed = df[col_name].apply(parse_json_like)
    else:
        df_parsed = df[col_name]
        
    df_expanded = df_parsed.apply(lambda x: pd.Series(x))
    try:
        df_expanded.drop([0], axis=1, inplace=True)
    except KeyError:
        pass

    if rename:
        df_expanded.columns = [col_name + "_" + str(key) for key in df_expanded.columns]
    
    return df_expanded 

In [19]:
#List of Dict 獨立成表
def extract_dict_lst_like(df, col_name, id_col):
    """
        col_name : 要處理的欄位
        id_col : 與主表連接的 id 

        回傳獨立的 dataframe
    """
    df_tmp = df.copy()
    #提出需要的欄位，並將重點欄位轉換為python物件
    df_current = df_tmp[[id_col, col_name]]
    df_current.columns = ['order_id', col_name] #以便區分訂單ID 以及這個表自己的id 
    #df_current[col_name] = df_current[col_name].apply(parse_json_like)

    #將 list of dicts 拆成多個row
    df_exploded = df_current.explode(col_name)

    #將每個 row 的 dict 獨立成自己的欄位 (與型態 2 一樣)
    detail_expanded = expand_dict_like(df_exploded, col_name, converted=True, rename=False)
    try:
        detail_expanded.drop(0, axis=1, inplace=True)
    except KeyError: #若沒有多餘的0欄位
        pass
    
    df_final = pd.concat([df_exploded['order_id'], detail_expanded], axis = 1)

    return df_final

### 資料清理

#### 1. 主表清理 + 儲存

In [20]:
df_parsed = df_raw.applymap(parse_json_like) #parse each data cells first
df = df_parsed.copy()
print("Finished parsing dataframes into python types.")

error_columns = [] #store columns that have not been converted successfully
subDF_dict = {}

loops = 0
while True:
    loops += 1
    flag = 0
    print(f"\n\nLoop : {loops}")
    columns = df.columns
    for col in columns:
        
        if col in error_columns: continue #若已經遇到錯誤則不繼續打開

        if df[col].isna().all(): #若欄位完全缺失則不需處理
            continue 
        else:
            sample = df[col].dropna().iloc[0] #尋找第一個非n/a的值，以此辨識處理方式

        #處理Dict類型
        if type(sample) == type(dict()):
            print(f"Expanding dict : {col}")
            flag = 1
            try:
                col_expanded = expand_dict_like(df, col, converted = True)
                df = pd.concat([df, col_expanded], axis=1)
                df.drop(col, axis=1, inplace=True)
            except:
                print(f"Problem processing {col} ... added to error_columns list")
                error_columns.append(col)

        #處理 List of Dicts 類型
        elif type(sample) == type(list()):
            print(f"Isolating a new subtable for column: {col}")
            
            try:
                expanded_df = extract_dict_lst_like(df, col, "_id_oid")
                subDF_dict[col] = expanded_df
                df.drop(col, axis=1, inplace=True)
            except:
                print(f"Problem processing {col} ... added to error_columns list")
                error_columns.append(col)

    if flag == 0:
        break

df.drop(["Unnamed: 0"], axis = 1, inplace=True)

Finished parsing dataframes into python types.


Loop : 1
Expanding dict : _id
Expanding dict : address
Expanding dict : chatRoom
Isolating a new subtable for column: customizations
Expanding dict : discount
Expanding dict : dispatching
Expanding dict : envInfo
Expanding dict : guidePurchase
Expanding dict : intercom
Isolating a new subtable for column: labels
Isolating a new subtable for column: memoImages
Expanding dict : newestStateTimes
Isolating a new subtable for column: notes
Expanding dict : originRef
Isolating a new subtable for column: processingImages
Expanding dict : quotation
Isolating a new subtable for column: refUnorderedReservations
Isolating a new subtable for column: referralFeeDiff
Expanding dict : referralFeeInfo
Expanding dict : satisfySurvey
Expanding dict : selfBookingInfo
Expanding dict : serviceCapacity
Expanding dict : serviceCharge
Expanding dict : serviceClassification
Isolating a new subtable for column: services
Isolating a new subtable for column: siteIm

c:\Users\joannelee\AppData\Local\anaconda3\envs\test_env\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  del sys.path[0]


Isolating a new subtable for column: dispatching_manuallyDispatchLogs
Isolating a new subtable for column: dispatching_selfBookingLogs
Expanding dict : originRef_coupon
Expanding dict : selfBookingInfo_copyOpt
Expanding dict : selfBookingInfo_fromUserBookedReservation


Loop : 3


In [21]:
#建立訂單類型資料夾
os.makedirs(f"{folder_name}", exist_ok=True)
#將主表存起來
df.to_csv(f"{folder_name}/main.csv", index=False)

#### 2. 副表清理 + 儲存

In [22]:
#清理獨立出的副表並儲存為獨立的.csv檔案

error_columns = [] #store columns that have not been converted successfully

for sub_table in subDF_dict.keys():
    print("\n=====\n")
    print(f"Cleaning sub_table {sub_table}")

    df_cleaned = subDF_dict[sub_table]
    df_cleaned.dropna(how = "all", axis = 1, inplace=True)
    cols = df_cleaned.columns

    loops = 0
    while True:
        loops += 1
        flag = 0
        print(f"Loop : {loops}")
        for col in df_cleaned.columns:
            
            if df_cleaned[col].isna().all(): #若欄位完全缺失則不需處理
                continue 
            else:
                sample = df_cleaned[col].dropna().iloc[0] #尋找第一個非n/a的值，以此辨識處理方式

            if type(sample) == type(dict()):
                print(f"Processing dict : {col}")
                flag = 1
                try:
                    col_expanded = expand_dict_like(df_cleaned, col, converted=True)
                    df_cleaned = pd.concat([df_cleaned, col_expanded], axis=1)
                    df_cleaned.drop(col, axis=1, inplace=True)
                except:
                    print(f"Problem processing {col} ... added to error_columns list")
                    error_columns.append(col)

        if flag == 0:
            break
    df_cleaned.to_csv(f"{folder_name}/{sub_table}.csv", index=False)
    print(f"Subtable saved to folder.")


=====

Cleaning sub_table customizations
Loop : 1
Processing dict : _id
Processing dict : itemId
Loop : 2
Subtable saved to folder.

=====

Cleaning sub_table labels
Loop : 1
Subtable saved to folder.

=====

Cleaning sub_table memoImages
Loop : 1
Subtable saved to folder.

=====

Cleaning sub_table notes
Loop : 1
Processing dict : _id
Processing dict : by


c:\Users\joannelee\AppData\Local\anaconda3\envs\test_env\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  del sys.path[0]


Loop : 2
Subtable saved to folder.

=====

Cleaning sub_table processingImages
Loop : 1
Subtable saved to folder.

=====

Cleaning sub_table refUnorderedReservations
Loop : 1
Subtable saved to folder.

=====

Cleaning sub_table referralFeeDiff
Loop : 1
Processing dict : by
Loop : 2
Subtable saved to folder.

=====

Cleaning sub_table services
Loop : 1
Processing dict : _id
Processing dict : serviceId
Loop : 2
Subtable saved to folder.

=====

Cleaning sub_table siteImages
Loop : 1
Subtable saved to folder.

=====

Cleaning sub_table stateLogs
Loop : 1
Processing dict : _id
Processing dict : by
Processing dict : newInfo
Processing dict : newState
Processing dict : originalState
Loop : 2
Subtable saved to folder.

=====

Cleaning sub_table vendersAssignedPriority
Loop : 1
Subtable saved to folder.

=====

Cleaning sub_table dispatching_manuallyDispatchCancelLogs
Loop : 1
Processing dict : _id
Processing dict : by
Processing dict : vender
Loop : 2
Subtable saved to folder.

=====

Cleanin

#### Cleaned Data Structures

In [23]:
print(f"資料夾名稱: {folder_name}")
print("\n======\n")

print("主表: main.csv\n")
print(f"副表列表 ({len(subDF_dict.keys())}):")

for key in subDF_dict.keys():
    print(f"-- {key}.csv")

print("\n=====\n")

print(f"主表副表連結: 主表的 _id_oid 欄位對上 副表的 order_id_oid 欄位")

資料夾名稱: unordered


主表: main.csv

副表列表 (14):
-- customizations.csv
-- labels.csv
-- memoImages.csv
-- notes.csv
-- processingImages.csv
-- refUnorderedReservations.csv
-- referralFeeDiff.csv
-- services.csv
-- siteImages.csv
-- stateLogs.csv
-- vendersAssignedPriority.csv
-- dispatching_manuallyDispatchCancelLogs.csv
-- dispatching_manuallyDispatchLogs.csv
-- dispatching_selfBookingLogs.csv

=====

主表副表連結: 主表的 _id_oid 欄位對上 副表的 order_id_oid 欄位


In [24]:
#把上面的文字寫進txt檔案方便未來了解
with open(f"{folder_name}\Description.txt", "w", encoding="utf-8") as f:
    f.write(f"資料夾名稱: {folder_name}\n")
    f.write("\n======\n\n")

    f.write("主表: main.csv\n\n")
    f.write(f"副表列表 ({len(subDF_dict.keys())}):\n")

    for key in subDF_dict.keys():
        f.write(f"-- {key}.csv\n")

    f.write("\n=====\n\n")

    f.write("主表副表連結: 主表的 _id_oid 欄位對上 副表的 order_id_oid 欄位\n")